In [1]:
import time
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn

from torch2trt import torch2trt

from model_training import get_loaders_and_classes, get_train_test_data, get_class_weights
from training_functions import *
from CNN import ErnNet
from utility_functions import *

In [3]:
import yaml
def read_params(config_path):
    with open(config_path) as yaml_file:
        config = yaml.safe_load(yaml_file)
    return config

config = read_params('settings.yaml')
batch_size = config['batch_size']
num_epochs = config['number_epochs']
# Extracting the training, validation and testing data
compressed_data_path = config['compressed_data_path']
data = decompress_data(compressed_data_path)

# Get data loaders
data_loaders_and_classes = get_loaders_and_classes(data, batch_size)

In [4]:
# Extracting the training, validation and testing data
x_train = data['x_train']
x_val = data['x_val']
x_test = data['x_test']

y_train = data['y_train']
y_val = data['y_val']
y_test = data['y_test']

In [9]:
# Residual network model
from utility_functions import qrelu, qconv2d

# def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1,norm=False,act=False,qat_mode=False,infer=False):
#     """3x3 convolution with padding"""
#     return qconv2d(in_channels=in_planes,out_channels=out_planes,kernel_size=3,stride=stride,groups=groups,padding=dilation,dilation=dilation,bias=False,act=act,norm=norm,qat=qat_mode,infer=infer)


class FirstLayer(nn.Module):
    """The layer of the network"""
    def __init__(self, num_channels, norm, act, qat_mode, infer):
        super(FirstLayer, self).__init__()

        # First convolutional layer of the model
        self.conv1 = qconv2d(in_channels=1, out_channels=num_channels, kernel_size=3, padding=1, bias=False, act=act, norm=norm, qat=qat_mode, infer=infer)

        self.relu = qrelu(inplace=True,qat=qat_mode,infer=infer)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):

        out = self.conv1(x)
        #out = self.bn1(out)
        out = self.relu(out)
        out = self.pool(out)
        
        return out

class Block(nn.Module):
    """A block that will be skipped over by the residual connectionS"""
    def __init__(self, num_channels, norm, act, qat_mode, infer):
        super(Block, self).__init__()
        self.num_channels = num_channels
        self.relu = qrelu(inplace=True,qat=qat_mode,infer=infer)
        
        # First layer of the block
        self.conv1 = qconv2d(in_channels=num_channels, out_channels=num_channels, kernel_size=1, padding=1, bias=False, act=act, norm=norm, qat=qat_mode, infer=infer)

        # Second layer of the block
        self.conv2 = qconv2d(in_channels=1, out_channels=num_channels, kernel_size=3, padding=1, bias=False, act=act, norm=norm, qat=qat_mode, infer=infer)

    def forward(self, x):
        identity = x

        out = self.conv1(x)

        out = self.relu(out)

        out = self.conv2(out)
        out = self.relu(out)

        out += identity # Creating the skip connection

        return out

class ErnNet(nn.Module): # Enerst Net
    def __init__(self, num_labels, norm, act, qat_mode, infer, num_channels=32):
        super(ErnNet, self).__init__()
        #Network = First layer -> block1 -> block2 ....
        self.first_layer = FirstLayer(num_channels,norm, act, qat_mode, infer)
        self.block1 = Block(num_channels, norm, act, qat_mode, infer)
        self.block2 = Block(num_channels, norm, act, qat_mode, infer)
        self.block3 = Block(num_channels, norm, act, qat_mode, infer)
        self.output_channels = self.block3.num_channels
    
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(self.output_channels * 1 * 1, num_labels)

    
    def forward(self, x):
        out = self.first_layer(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.avgpool(out)
        out = out.view(-1, self.output_channels * 1 * 1)
        out = self.fc1(out)

        return out





In [12]:
model = ErnNet(num_labels=6, norm=True, act=True, qat_mode=True, infer=True)

NameError: name 'num_channels' is not defined

In [4]:
# Calibrating model

random_idxs_train = np.random.randint(len(x_train), size=100)
random_idxs_val = np.random.randint(len(x_val), size=100)
random_idxs_test = np.random.randint(len(x_test), size=100)

random_examples_train = x_train[random_idxs_train]
random_examples_val = x_val[random_idxs_val]
random_examples_test = x_train[random_idxs_test]

int_8_calib_dataset = torch.from_numpy(np.concatenate((random_examples_train, random_examples_test, random_examples_val)))


In [5]:
x = torch.rand((batch_size, 1, 128, 45)).cuda()
# create some regular pytorch model...
network = ErnNet()
checkpoint = torch.load('checkpoint.pt')
network.load_state_dict(checkpoint)
network.eval().cuda()

# Creating TensorRT models
model_trt = torch2trt(network, [x], max_batch_size=batch_size)
model_trt_fp16 = torch2trt(network, [x], fp16_mode=True, max_batch_size=batch_size)
model_trt_int8 = torch2trt(network, [x], int8_mode=True, max_batch_size=batch_size, int8_calib_dataset=int_8_calib_dataset)


/home/nyasha/anaconda3/envs/thesis/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [6]:
# Initialising training parameters
class_weights = get_class_weights(y_train, 'cuda')
criterion = nn.CrossEntropyLoss(weight=class_weights)

/home/nyasha/anaconda3/envs/thesis/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=['2_walking' 'clutter/noise' 'running' 'sphere_swing' 'vehicle' 'walking'], y=['2_walking' 'sphere_swing' 'vehicle' ... 'clutter/noise' 'running'
 'sphere_swing'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [7]:
def evaluate_model(model_on_device, data_loader, criterion, classes, show_cm=False):

    """Evaluate Performance on test set"""
    model_on_device.eval()  # Turn off gradient computations
    num_batches = len(data_loader)
    correct = 0
    total = 0
    running_loss = 0
    y_tot = torch.empty(0)
    y_pred_tot = torch.empty(0)

    with torch.no_grad():
        t0 = time.time()
        for data in data_loader:
            images, labels = data
            images = images.to('cuda').float()
            labels = labels.to('cuda')
            outputs = model_on_device(images)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            running_loss += loss.item()

            labels = labels.cpu()
            predicted = predicted.cpu()

            y_tot = torch.cat((y_tot, labels), 0)
            y_pred_tot = torch.cat((y_pred_tot, predicted), 0)
        t1 = time.time()
    accuracy = 100 * correct / total
    accuracy = 100 * correct / total
    errors = y_pred_tot - y_tot != 0
    y_pred_errors = y_pred_tot[errors]
    y_true_errors = y_tot[errors]

    # Plotting the Confusion Matrix
    if show_cm:
        generate_confusion_matrix(classes, y_tot, y_pred_tot)

    print(f'Time taken on inference is {(t1-t0)/num_batches}')

    return running_loss / num_batches, accuracy, errors, y_pred_errors, y_true_errors


def generate_confusion_matrix(classes, y_tot, y_pred_tot):
    cm = confusion_matrix(y_tot.numpy(), y_pred_tot.numpy())
    num_classes = len(classes)
    np.set_printoptions(precision=4)

    # Coloured confusion matrix
    plt.figure(figsize=(12, 12))
    cm = confusion_matrix(y_tot.numpy(), y_pred_tot.numpy(), normalize="true")
    plt.imshow(cm, cmap=plt.cm.Blues)

    for (i, j), z in np.ndenumerate(cm):
        plt.text(j, i, "{:0.3f}".format(z), ha="center", va="center")

    plt.xticks(range(num_classes))
    plt.yticks(range(num_classes))
    plt.xlabel("Prediction")
    plt.ylabel("True")

    plt.gca().set_xticklabels(classes)
    plt.gca().set_yticklabels(classes)

    plt.title("Normalized Confusion Matrix for the Data")
    plt.colorbar()
    plt.show()

In [9]:
train_loader = data_loaders_and_classes['train_loader']
val_loader = data_loaders_and_classes['val_loader']
test_loader = data_loaders_and_classes['test_loader']
classes = data_loaders_and_classes['classes']

In [10]:
# Pytorch model
_, accuracy, errors, y_pred_errors, y_true_errors = evaluate_model(network, val_loader, criterion, classes, show_cm=False)
print(accuracy)

Time taken on inference is 0.004473873849451189
91.81851989587207


In [11]:
# TRT model FP32
_, accuracy, errors, y_pred_errors, y_true_errors = evaluate_model(model_trt, val_loader, criterion, classes, show_cm=False)
print(accuracy)

Time taken on inference is 0.002103014105170436
91.81851989587207


In [12]:
# TRT model FP16
_, accuracy, errors, y_pred_errors, y_true_errors = evaluate_model(model_trt_fp16, val_loader, criterion, classes, show_cm=False)
print(accuracy)

Time taken on inference is 0.0015588427436422315
92.07883971736705


In [13]:
# TRT Model INT8
_, accuracy, errors, y_pred_errors, y_true_errors = evaluate_model(model_trt_int8, val_loader, criterion, classes, show_cm=False)
print(accuracy)

Time taken on inference is 0.0013057973963269114
88.88062476757159


In [15]:
network.load_state_dict(checkpoint)

<All keys matched successfully>

In [16]:
torch.save(network.state_dict(), 'trt_models/pytorch_model.pth')
torch.save(model_trt.state_dict(), 'trt_models/model_trt_fp32.pth')
torch.save(model_trt_fp16.state_dict(), 'trt_models/model_trt_fp16.pth')
torch.save(model_trt_int8.state_dict(), 'trt_models/model_trt_int8.pth')
